# Generating counterfactual explanations with any ML model

The goal of this notebook is to show how to generate CFs for ML models using frameworks other than TensorFlow or PyTorch. This is a work in progress and here we show a method to generate diverse CFs by two methods: 
1. Independent random sampling of features
2. Genetic algorithm

We use scikit-learn models for demonstration.  

# 1. Independent random sampling of features

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

## Loading dataset

We use the "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as described in dice_ml.utils.helpers module.

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

## Training a custom ML model

Below, we build an Artificial Neural Network using *MLPClassifier* in scikit-learn. We try to use the same set of parameters as used in this advanced [notebook](DiCE_with_private_data.ipynb), however, there are other framework-dependent parameters that can't be easily ported, so the accuracy/performance of the two models will be different.

In [5]:
train, test = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']
X_test = test.loc[:, test.columns != 'income']
y_test = test.loc[:, test.columns == 'income']

In [6]:
mlp = MLPClassifier(hidden_layer_sizes=(20), alpha=0.001, learning_rate_init=0.01, batch_size=32, random_state=17,
                    max_iter=20, verbose=False, validation_fraction=0.2, ) #max_iter is epochs in TF
mlp.fit(X_train, y_train.values.ravel())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.001, batch_size=32, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=20, learning_rate='constant',
              learning_rate_init=0.01, max_iter=20, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=17, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.2, verbose=False, warm_start=False)

In [7]:
# provide the trained ML model to DiCE's model object
backend = None
m = dice_ml.Model(model=mlp, backend=backend)

## Generate diverse counterfactuals

In [8]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [11]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.002518



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,47.0,Self-Employed,Some-college,Married,Professional,-,-,19.0,0.674
1,66.0,Government,Prof-school,Separated,Other/Unknown,Other,Male,81.0,0.71
2,33.0,Government,Doctorate,Separated,Sales,Other,Male,50.0,0.521
3,66.0,-,Assoc,Married,Professional,-,-,64.0,0.547


It can be observed that the random sampling method produces less sparse CFs in contrast to current DiCE's implementation. The sparsity issue with random sampling worsens with increasing *total_CFs* 

Further, different sets of counterfactuals can be generated with different random seeds.

In [12]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", random_seed=9) # default ranomd see is 17

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [13]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.002518



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,76.0,Self-Employed,Some-college,Married,Professional,-,-,-,0.644
1,69.0,Government,Prof-school,Separated,Sales,Other,Male,57.0,0.533
2,63.0,Government,Doctorate,Separated,Sales,Other,Male,80.0,0.657
3,33.0,Self-Employed,Some-college,Married,Professional,-,-,68.0,0.694


### Selecting the features to vary

When few features are fixed, random sampling is unable to generate valid CFs while we get valid diverse CFs with current DiCE.

In [14]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       features_to_vary=['workclass','education','occupation','hours_per_week'])

Only 0 (required 4) Diverse Counterfactuals found for the given configuation, perhaps try with different values of proximity (or diversity) weights or learning rate... ; total time taken: 00 min 00 sec


In [15]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.002518



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Assoc,-,White-Collar,-,-,83.0,0.031
1,-,Self-Employed,Some-college,-,-,-,-,22.0,0.002
2,-,Other/Unknown,-,-,Blue-Collar,-,-,50.0,0.003
3,-,Other/Unknown,-,-,Blue-Collar,-,-,69.0,0.006


### Choosing feature ranges

Since the features are sampled randomly, they can freely vary across their range. In the below example, we show how range of continuous features can be controlled using *permitted_range* parameter that can now be passed during CF generation.

In [16]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite",
                                       permitted_range={'age':[22,50],'hours_per_week':[40,60]})

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [17]:
dice_exp.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,24.0,Private,HS-grad,Single,Service,White,Female,49.0,0.002518



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,26.0,Government,Doctorate,Separated,Sales,Other,Male,51.0,0.507
1,31.0,Government,Doctorate,Separated,Sales,Other,Male,52.0,0.579
2,28.0,Self-Employed,Some-college,Married,Professional,-,-,-,0.691
3,30.0,-,Assoc,Married,Professional,-,-,-,0.624


# 2. Genetic Algorithm

Here, we show how to use DiCE can be used to generate CFs for any ML model by using the genetic algorithm to find the best counterfactuals close to the query point. The genetic algorithm converges quickly, and promotes diverse counterfactuals. 

We label-encode the data instead of one-hot encoding here. We eventually hope to replace label-encoding with a more meaningful transformation for categorical values to numerical values

In [18]:
train, test = d.split_data(d.normalize_data(d.label_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']
X_test = test.loc[:, test.columns != 'income']
y_test = test.loc[:, test.columns == 'income']

In [19]:
mlp = MLPClassifier(hidden_layer_sizes=(20), alpha=0.001, learning_rate_init=0.01, batch_size=32, random_state=17,
                    max_iter=20, verbose=False, validation_fraction=0.2, ) #max_iter is epochs in TF
mlp.fit(X_train, y_train.values.ravel())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.001, batch_size=32, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=20, learning_rate='constant',
              learning_rate_init=0.01, max_iter=20, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=17, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.2, verbose=False, warm_start=False)

In [20]:
# provide the trained ML model to DiCE's model object
backend = None
m = dice_ml.Model(model=mlp, backend=backend)

In [21]:
from dice_ml.explainer_interfaces.dice_genetic import DiceGenetic

In [22]:
# initiate DiceGenetic
exp_genetic = DiceGenetic(d, m)

In [23]:
# generate counterfactuals
dice_exp_genetic = exp_genetic.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 00 sec


In [24]:
dice_exp_genetic.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,24.0,Private,HS-grad,Single,Service,White,Female,49.0,0.005071



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,31.0,Other/Unknown,Assoc,Married,Blue-Collar,-,Male,50.0,0.617
1,33.0,Other/Unknown,Bachelors,Divorced,Blue-Collar,Other,-,-,0.611
2,31.0,Other/Unknown,Assoc,Married,Other/Unknown,Other,Male,-,0.685
3,30.0,Government,Assoc,Married,Blue-Collar,Other,Male,48.0,0.607


Additional tools such as selecting features to vary and choosing feature ranges will be available shortly